In [2]:
!pip3 install -q tensorflow_decision_forests

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.4.1 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tensorflow-gpu 2.4.1 requires grpcio~=1.32.0, but you have grpcio 1.39.0 which is incompatible.
tensorflow-gpu 2.4.1 requires h5py~=2.10.0, but you have h5py 3.1.0 which is incompatible.
tensorflow-gpu 2.4.1 requires tensorflow-estimator<2.5.0,>=2.4.0, but you have tensorflow-estimator 2.6.0 which is incompatible.


In [3]:
!pip install wurlitzer

In [20]:
import tensorflow_decision_forests as tfdf
import numpy as np
import pandas as pd
import tensorflow as tf


In [21]:
# Check the version of TensorFlow Decision Forests
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

Found TensorFlow Decision Forests v0.1.9


In [22]:
# Read in the data
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv',index_col=0)
test  = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv', index_col=0)

In [23]:
train['nan_count']=train.isnull().sum(axis=1)
test['nan_count']=test.isnull().sum(axis=1)

In [24]:
xtrain = train.sample(frac=0.2, random_state=0)
valid = train.drop(xtrain.index).sample(frac=0.05, random_state=0)

,0,1
claim,96142,95442


In [ ]:
xtrain['claim'].value_counts().to_frame().T

In [25]:

valid['claim'].value_counts().to_frame().T

,1,0
claim,19219,19098


In [26]:
xtrain=xtrain.fillna(0)
valid_data=valid.fillna(0)
test=test.fillna(0)

In [27]:
# Convert the dataset into a TensorFlow dataset.
train = tfdf.keras.pd_dataframe_to_tf_dataset(xtrain, label='claim')
valid = tfdf.keras.pd_dataframe_to_tf_dataset(valid, label='claim')
test = tfdf.keras.pd_dataframe_to_tf_dataset(test)

In [ ]:
%%time

# Train a Random Forest model.
model = tfdf.keras.RandomForestModel()

# Train a Gradient Boosted Trees model.
#model = tfdf.keras.GradientBoostedTreesModel(num_trees=500)
model.fit(train)

2994/2994 [==============================] - 27s 9ms/step


In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt
logs = model.make_inspector().training_logs()

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")
plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")
plt.show()

In [ ]:
predictions = model.predict(valid)
y_true      = valid_data["claim"]

from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y_true, predictions)
print("The ROC AUC score is %.5f" % ROC_AUC )

In [32]:
sample = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sample['claim'] = model.predict(test)

In [33]:
sample.to_csv('submission.csv',index=False)